<a href="https://colab.research.google.com/github/CorgitoErgo/Arduino-uart-imu/blob/main/cc11_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 11 Spark

Author: ISTD, SUTD

Title: Lab 11, Spark part 1

Date: March 5, 2025

## Learning outcome


By the end of this lesson, you are able to

* Submit PySpark jobs to a Spark cluster
* Paralelize data processing using PySpark


You can either execute this lab directly on the aws cluster with HDFS file system, or you can install PySpark in Google Colab and load the files locally. The main difference in coding is that we do not load the context from the HDFS filesystem, but instead just load a local file. Other than than that, all PySpark commands are the same.

To run this lab, you can make a copy of this notebook or `File -> Open in Playground Mode`.

## Installing PySpark in Google Colab

To install PySpark in Google Collab, execute the below cell. This will download Spark and install all necessary libraries for this lab.

In [58]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
35 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

## Wordcount Example

Let us first download the necessary data file. We can find it at `https://raw.githubusercontent.com/istd50043-2023-spring/cohort_problems/main/cc11/ex1/data.csv`.

Colab lets us execute unix commands, as long as we prepend them with `!`. So let's download the file and move it into a new folder called `input`. While we are at it, let's create a folder called `output` as well.

In [59]:
!wget https://raw.githubusercontent.com/sutd50043/cohortclass/main/cc10/data/TheCompleteSherlockHolmes.txt
!mkdir input
!mv TheCompleteSherlockHolmes.txt input/
!mkdir output
!ls

--2025-08-01 02:12:24--  https://raw.githubusercontent.com/sutd50043/cohortclass/main/cc10/data/TheCompleteSherlockHolmes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3705628 (3.5M) [text/plain]
Saving to: ‘TheCompleteSherlockHolmes.txt’

TheCompleteSherlock 100%[===================>]   3.53M  10.0MB/s    in 0.4s    

2025-08-01 02:12:25 (10.0 MB/s) - ‘TheCompleteSherlockHolmes.txt’ saved [3705628/3705628]

mkdir: cannot create directory ‘input’: File exists
mkdir: cannot create directory ‘output’: File exists
drive  input  output  sample_data


In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You can check that the data.csv file downloaded by uncollapsing the left panel and checking the folder contents.

Now we are ready to write our PySpark code. The goal is to write a simple wordcounter:

In [63]:
import sys
from pyspark import SparkContext, SparkConf

# sc.stop() # uncomment this during debugging to restart your context in case execution stopped mid-way this cell.

conf = SparkConf().setAppName("Wordcount Application")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

# note that we load the text file directly with a local path instead of providing an hdfs url
input_file_name = 'input/TheCompleteSherlockHolmes.txt'
text_file = sc.textFile(input_file_name)

counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

output_folder = './output/wordcount'
counts.saveAsTextFile(output_folder)

sc.stop()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Wordcount Application, master=local[*]) created by __init__ at /tmp/ipython-input-628446628.py:7 

## Exercise 1

Write a PySpark application which takes a (set of) Comma-seperated-value (CSV) file(s) with 2 columns and output a CSV file with first two columns same as the input file, and the third column contains the values obtained by splitting the first column using the second column as delimiter.

The input file can be found here: `https://raw.githubusercontent.com/sutd50043/cohortclass/main/cc11/ex1/data.csv`.

For example, given input from a file:

```
50000.0#0#0#,#
0@1000.0@,@
1$,$
1000.00^Test_string,^
```


the program should output

```
50000.0#0#0#,#,['50000.0', '0', '0']
0@1000.0@,@,['0', '1000.0', '']
1$,$,['1', '']
1000.00^Test_string,^,['1000.00', 'Test_string']
```

and write it to a file.



In [68]:
import sys
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.appName("ex1").getOrCreate()
sc = spark.sparkContext

input_rdd = sc.textFile("./ex1_input/data.csv")
print(input_rdd.collect())
formatted_rdd = input_rdd.map(lambda line: line.strip().split(",")).map(lambda cols: cols[0].split(cols[1]))

zipped_rdd = input_rdd.zip(formatted_rdd).map(lambda x: f"{x[0]},{x[1]}")

output_folder = "./ex1_output"
if os.path.exists(output_folder):
  !rm -rf {output_folder}

zipped_rdd.saveAsTextFile(output_folder)
sc.stop()

['50000.0#0#0#,#', '0@1000.0@,@', '1$,$', '1000.00^Test_string,^']


## Exercise 2

Write PySpark application which aggregates (counts) a (set of) CSV file(s) with 4 columns based on its third column, the destination IP.

The input file can be found here: `https://raw.githubusercontent.com/sutd50043/cohortclass/main/cc11/ex2/data.csv`

Given input

```
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604900, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604900, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604900, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604900, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604900, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604900, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604899, 10.0.0.2.54880, 10.0.0.3.5001, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
05:49:56.604908, 10.0.0.3.5001, 10.0.0.2.54880, 2
```
the program should output

```
 10.0.0.3.5001,13
 10.0.0.2.54880,7
```

In [96]:
import sys
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.appName("ex2").getOrCreate()
sc = spark.sparkContext

input_rdd = sc.textFile("./ex2_input/data.csv")
#print(input_rdd.collect())
#formatted_rdd = input_rdd.map(lambda line: line.strip().split(",")).map(lambda cols: cols[0].split(cols[1]))
#print(formatted_rdd.collect())

formatted_rdd = input_rdd.map(lambda line: line.strip().split(",")).map(lambda cols: cols[2].strip())
#print(formatted_rdd.collect())

count_rdd = formatted_rdd.map(lambda x: (x, 1)).reduceByKey(lambda a,b: a+b)

zipped_rdd = count_rdd.map(lambda x: f"{x[0]},{x[1]}")
print(zipped_rdd.collect())

#zipped_rdd = input_rdd.zip(formatted_rdd).map(lambda x: f"{x[0]},{x[1]}")

output_folder = "./ex2_output"
if os.path.exists(output_folder):
  !rm -rf {output_folder}

zipped_rdd.saveAsTextFile(output_folder)
sc.stop()

['10.0.0.3.5001,13', '10.0.0.2.54880,7']


## Exercise 3

Given the same input as Exercise 2, write a PySpark application which outputs the following:

```
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604900,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604900,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604900,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604900,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604900,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604900,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604899,10.0.0.2.54880, 10.0.0.3.5001, 2, 13
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
05:49:56.604908, 10.0.0.3.5001,10.0.0.2.54880, 2, 7
```


In the event the input is very huge with too many unique destination IP values, can your program scale?


The questions were adopted from `https://jaceklaskowski.github.io/spark-workshop/exercises/`


In [94]:
import sys
from pyspark import SparkContext, SparkConf
spark = SparkSession.builder.appName("ex2").getOrCreate()
sc = spark.sparkContext

input_rdd = sc.textFile("./ex2_input/data.csv")
#print(input_rdd.collect())
#formatted_rdd = input_rdd.map(lambda line: line.strip().split(",")).map(lambda cols: cols[0].split(cols[1]))
#print(formatted_rdd.collect())

formatted_rdd = input_rdd.map(lambda line: line.strip().split(",")).map(lambda cols: cols[2].strip())
#print(formatted_rdd.collect())

count_rdd = formatted_rdd.map(lambda x: (x,1)).reduceByKey(lambda a,b: a+b)
print(count_rdd.collect())

zipped_rdd = count_rdd.map(lambda x: f"{x[0]},{x[1]}")
#print(zipped_rdd.collect())

joined_rdd = zipped_rdd.join(input_rdd).collect()
#print(joined_rdd)

#zipped_rdd = input_rdd.zip(formatted_rdd).map(lambda x: f"{x[0]},{x[1]}")

output_folder = "./ex3_output"
if os.path.exists(output_folder):
  !rm -rf {output_folder}

zipped_rdd.saveAsTextFile(output_folder)
sc.stop()

[('10.0.0.3.5001', 13), ('10.0.0.2.54880', 7)]
